In [1]:
from random import sample
import json
import base64
import gzip
from io import BytesIO
import pandas as pd
import numpy as np
from PIL import Image
from tqdm.notebook import tqdm
from sqlalchemy import select, func
from eyened_orm import (
    ImageInstance,
    Annotation,
    AnnotationData,
    AnnotationType,
    DBManager,
)
from eyened_orm.utils import get_config
from eyened_orm.utils.zarr.manager_annotation import AnnotationZarrStorageManager

In [2]:
config = get_config("config.prod")
DBManager.init(config)
session = DBManager.get_session()
annotation_zarr_storage_manager = AnnotationZarrStorageManager(
    config.annotations_zarr_store
)

In [3]:
def get_annotations_with_annotation_type(annotation_type_ids):
    # 
    all_annots = session.execute(
        select(Annotation, AnnotationData, ImageInstance)
        .join_from(Annotation, AnnotationData, isouter=True)
        .join_from(Annotation, ImageInstance, isouter=True)
        .where(~Annotation.Inactive & (Annotation.AnnotationTypeID.in_(annotation_type_ids)))
        .order_by(func.random())
        .limit(1000)
    ).all()
    all_annots = [(annot, annot_data, image_instance) for annot, annot_data, image_instance in all_annots if annot_data is not None]
    return all_annots

In [4]:

# resolutions = []
# for annot, annot_data, image_instance in tqdm(get_annotations_with_annotation_type([14])):
#     # if annot_data is None
#     assert annot_data.path.suffix == ".png", annot_data.path

#     res_db = (image_instance.Columns_x, image_instance.Rows_y)
#     resolutions.append(res_db)

#     im = Image.open(annot_data.path)
#     rs_im = im.size

#     if res_db != rs_im:
#         print(f'Found shape {rs_im} != {res_db} for {annot_data.path}')
#         continue

#     im = np.array(im)
#     im = im[...,None,None].astype(np.uint8)

#     if len(im.shape) != 4:
#         print(f'Found shape {im.shape} for {annot_data.path}')
#         continue

#     annotation_zarr_storage_manager.write('test', im, None)

In [6]:
# 13 binary mask annotations
# 14 probability annotations
# 20 oct binary mask annotations
# 4 label numbers
def open_data(dpath, db_res):
    if dpath.suffix == ".gz":
        with gzip.open(dpath, 'rb') as f:
            im = np.frombuffer(f.read(), dtype=np.uint8)
            im = im.reshape(db_res)
    else:
        im = Image.open(dpath)

        if db_res[:2] != im.size:
            raise RuntimeError(f'Found shape {im.size} != {db_res} for {annot_data.path}')

        if im.mode != "L":
            print(f"Found mode {im.mode} for {annot_data.path}, annot_id: {annot.AnnotationID}")
            im = im.convert('L')

        im = np.array(im)
        
    if len(im.shape) == 2:
        im = im[...,None,None].astype(np.uint8)
    elif len(im.shape) == 3:
        im = im[...,None].astype(np.uint8)
    else:
        raise RuntimeError(f'Found shape {im.shape} for {annot_data.path}')

    return im

for annot, annot_data, image_instance in get_annotations_with_annotation_type([20]):
    # if annot_data is None
    # assert annot_data.path.suffix == ".png", annot_data.path

    res_db = (image_instance.Rows_y, image_instance.Columns_x, image_instance.NrOfFrames)

    try:
        im = open_data(annot_data.path, res_db)
    except Exception as e:
        print(f'Error opening {annot_data.path}: {e}')
        continue
    
    # print(im.shape)
    if len(im.shape) != 4:
        print(f'Found shape {im.shape} for {annot_data.path}')
        continue

    annotation_zarr_storage_manager.write('test', im, None)

KeyboardInterrupt: 

In [32]:
57999360  / (885*512)

128.0

In [18]:
d=open_data('/mnt/oogergo/eyened/eyened_platform/annotations/1691382/421525_0.binary.gz')

In [19]:
d.shape

(92184576,)

In [ ]:


# from matplotlib import pyplot as plt
# # test A/V conversion
# annots = get_annotations_with_annotation_type([9])
# annot = annots[10]
# img_size = (annot[2].Columns_x, annot[2].Rows_y)
# with open(annot[1].path, 'r') as f:
#     data = json.load(f)
# for branch in data['branches']:
#     drawing = branch['drawing'][22:]
#     drawing = base64.b64decode(drawing)
#     # drawing = np.frombuffer(drawing, dtype=np.uint8)
#     drawing = Image.open(BytesIO(drawing))
#     drawing = np.array(drawing)[:, :, 0]
#     drawing = drawing.reshape(img_size[1], img_size[0])
#     plt.imshow(drawing)

In [ ]:
# artery-vein annotations
import json
import base64
resolutions = []
for annot, annot_data, image_instance in get_annotations_with_annotation_type([9]):
    # if annot_data is None
    assert annot_data.path.suffix == ".json", annot_data.path

    if image_instance is None:
        print(f"Found image_instance is None for {annot_data.path}, annot_id: {annot.AnnotationID}")
        continue

    with open(annot_data.path, 'r') as f:
        data = json.load(f)

    mask = Annotation.by_id(data['mask_id'])
    if mask is None:
        print(f"Found mask is None for {annot_data.path}, annot_id: {annot.AnnotationID}")
        continue

    img_size = (annot[2].Columns_x, annot[2].Rows_y)
    with open(annot[1].path, 'r') as f:
        data = json.load(f)
    for branch in data['branches']:
        drawing = branch['drawing'][22:]
        drawing = base64.b64decode(drawing)
        # drawing = np.frombuffer(drawing, dtype=np.uint8)
        drawing = Image.open(BytesIO(drawing))
        drawing = np.array(drawing)[:, :, 0]
        drawing = drawing.reshape(img_size[1], img_size[0])

        # put in Zarr
        annot_index = annotation_zarr_storage_manager.write('test', drawing, None)

        # make and link new annotations
        new_annot = Annotation(
            AnnotationTypeID=13,
            ImageInstanceID=image_instance.ImageInstanceID,
            AnnotationReferenceID=mask.AnnotationID,
            # AnnotationDataID=annot_data.AnnotationDataID,
        )

    # res_db = (image_instance.Columns_x, image_instance.Rows_y)
    # resolutions.append(res_db)

    # im = Image.open(annot_data.path)
    # rs_im = im.size

    # if res_db != rs_im:
    #     print(f'Found shape {rs_im} != {res_db} for {annot_data.path}')
    #     continue

    # if im.mode not in ["RGB", "RGBA"]:
    #     print(f"Found mode {im.mode} for {annot_data.path}, annot_id: {annot.AnnotationID}")
    #     im = im.convert('RGB')

    # im = np.array(im)
    # im = im[...,:2] # red and green channels
    # im = im.reshape(1, im.shape[0], im.shape[1], 1, im.shape[2])

Found image_instance is None for /mnt/oogergo/eyened/eyened_platform/annotations/385966/24268_0.json, annot_id: 24268


In [7]:
# R/G masks
resolutions = []
for annot, annot_data, image_instance in get_annotations_with_annotation_type([2,3,5]):
    # if annot_data is None
    assert annot_data.path.suffix == ".png", annot_data.path

    if image_instance is None:
        print(f"Found image_instance is None for {annot_data.path}, annot_id: {annot.AnnotationID}")
        continue

    res_db = (image_instance.Columns_x, image_instance.Rows_y)
    resolutions.append(res_db)

    im = Image.open(annot_data.path)
    rs_im = im.size

    if res_db != rs_im:
        print(f'Found shape {rs_im} != {res_db} for {annot_data.path}')
        continue

    if im.mode not in ["RGB", "RGBA"]:
        print(f"Found mode {im.mode} for {annot_data.path}, annot_id: {annot.AnnotationID}")
        im = im.convert('RGB')

    im = np.array(im)
    im = im[...,:2] # red and green channels
    im = im.reshape(1, im.shape[0], im.shape[1], 1, im.shape[2])
    # im = im[...,None,None].astype(np.uint8)

    # print(im.shape)
    # if len(im.shape) != 4:
    #     print(f'Found shape {im.shape} for {annot_data.path}')
    #     continue

    # annotation_zarr_storage_manager.write('test', im, None)

Found mode L for /mnt/oogergo/eyened/eyened_platform/annotations/macula3/2195243.png, annot_id: 677239
Found mode L for /mnt/oogergo/eyened/eyened_platform/annotations/macula3/2217629.png, annot_id: 671653
Found mode L for /mnt/oogergo/eyened/eyened_platform/annotations/macula3/2204141.png, annot_id: 668494
Found mode L for /mnt/oogergo/eyened/eyened_platform/annotations/macula3/2212353.png, annot_id: 677102
Found mode L for /mnt/oogergo/eyened/eyened_platform/annotations/macula3/2225498.png, annot_id: 672951
Found mode L for /mnt/oogergo/eyened/eyened_platform/annotations/macula2/2214117.png, annot_id: 173429
Found mode L for /mnt/oogergo/eyened/eyened_platform/annotations/macula3/2204055.png, annot_id: 676018
Found mode L for /mnt/oogergo/eyened/eyened_platform/annotations/macula3/2188185.png, annot_id: 668926
Found mode L for /mnt/oogergo/eyened/eyened_platform/annotations/macula3/2218258.png, annot_id: 671069
Found mode L for /mnt/oogergo/eyened/eyened_platform/annotations/macula3/